In [1]:
import pandas as pd
import numpy as np

from src.data_cleaning import data_labeling
from src.data_cleaning import feature_engineering
from src.language_processing import to_tfidf
from src.modeling import test_model

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split,KFold

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
df = data_labeling('data/data_prepping_zeros_index.xlsx',
              'data/smalleyeball_partone.xlsx',
              'data/smalleyeball_parttwo.xlsx')

/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['spam_corporate'][df.screen_name.isin(combined_lst)] = 1
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['spam_bot'][df.expanded_url == 'https://acrobat.adobe.com'] = 1
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [3]:
df2 = feature_engineering(df)

/home/daniel/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df3['time_created'] = pd.to_datetime(df3['created_at']).copy()
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [17]:
df2 = feature_engineering(df)
df3 = df2
text_tfidf_matrix = to_tfidf(df3,'description',2000)
col_name_lst = []
for n in range(text_tfidf_matrix.shape[1]):
    col_name = f'text-TF-IDF-{n}'
    df3[col_name] = text_tfidf_matrix[:, n]
    col_name_lst.append(col_name)
data_training_set = df3[:16000]
data_test_set = df3[16000:]
y = data_training_set['spam_marketing']
X = data_training_set[['Docusign', 'onespan', 'signnow','adobe sign','listed_count', 'statuses_count','followers_count',
          'favourites_count', 'friends_count','time_float_sin','time_float_cos', 'is_description_none']+col_name_lst]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
model = RandomForestClassifier()
model.fit(X_train,y_train)

y_predict = model.predict(X_test)
y_pred = model.predict_proba(X_test)
print("Accuracy score of this model is" + "\n")
print(accuracy_score(y_test,y_predict))
print("\n" + "Confusion Matrix of this model is" + "\n")
print(confusion_matrix(y_test,y_predict))
print("\n" + "Log-Loss score of this model is" + "\n")
print(log_loss(y_test,y_pred))
print("\n" + "AUC score of this model is" + "\n")
print(roc_auc_score(y_test,y_predict))
y = data_test_set['spam_marketing']
X = data_test_set[['Docusign', 'onespan', 'signnow','adobe sign','listed_count', 'statuses_count','followers_count',
          'favourites_count', 'friends_count','time_float_sin','time_float_cos', 'is_description_none']+col_name_lst]
y_predict = model.predict(X)
y_pred = model.predict_proba(X)
print("\n" + "Accuracy score of this model is" + "\n")
print(accuracy_score(y,y_predict))
print("\n" + "Confusion Matrix of this model is" + "\n")
print(confusion_matrix(y,y_predict))
print("\n" + "Log-Loss score of this model is" + "\n")
print(log_loss(y,y_pred))
print("\n" + "AUC score of this model is" + "\n")
print(roc_auc_score(y,y_predict))

/home/daniel/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df3['time_created'] = pd.to_datetime(df3['created_at']).copy()
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

Accuracy score of this model is

0.96125

Confusion Matrix of this model is

[[3789   29]
 [ 126   56]]

Log-Loss score of this model is

0.33894481790428804

AUC score of this model is

0.6500483539509208

Accuracy score of this model is

0.8684473290064902

Confusion Matrix of this model is

[[3439  137]
 [ 390   40]]

Log-Loss score of this model is

0.9868739264470417

AUC score of this model is

0.5273561469226367


In [6]:
model = MultinomialNB()
df3 = df2

X = to_tfidf(df3,'text',3000)
y = df3['spam_bot']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

model.fit(X_train,y_train)

y_predict = model.predict(X_test)
y_pred = model.predict_proba(X_test)
print("Accuracy score of this model is" + "\n")
print(accuracy_score(y_test,y_predict))
print("\n" + "Confusion Matrix of this model is" + "\n")
print(confusion_matrix(y_test,y_predict))
print("\n" + "Log-Loss score of this model is" + "\n")
print(log_loss(y_test,y_pred))
print("\n" + "AUC score of this model is" + "\n")
print(roc_auc_score(y_test,y_predict))

Accuracy score of this model is

0.6399440223910435

Confusion Matrix of this model is

[[ 432 1657]
 [ 144 2769]]

Log-Loss score of this model is

0.5992726361457885

AUC score of this model is

0.5786819685676381


In [8]:
df2[['text_without_accent', 'text_word_tokens', 'processed_text', 'stemmed','ngrammed']]

,text_without_accent,text_word_tokens,processed_text,stemmed,ngrammed
0,just closed a deal in 27 hours using #AdobeSig...,"[just, closed, a, deal, in, 27, hours, using, ...","[closed, deal, 27, hours, using, AdobeSign, ht...","[close, deal, 27, hour, use, adobesign, httpst...","[close, deal, 27, hour, use, adobesign, httpst..."
1,just closed a deal in 2 days using #AdobeSign ...,"[just, closed, a, deal, in, 2, days, using, #,...","[closed, deal, 2, days, using, AdobeSign, http...","[close, deal, 2, day, use, adobesign, httpstco...","[close, deal, 2, day, use, adobesign, httpstco..."
2,just closed a deal in 2 hours using #AdobeSign...,"[just, closed, a, deal, in, 2, hours, using, #...","[closed, deal, 2, hours, using, AdobeSign, htt...","[close, deal, 2, hour, use, adobesign, httpstc...","[close, deal, 2, hour, use, adobesign, httpstc..."
3,just closed a deal in 26 hours using #AdobeSig...,"[just, closed, a, deal, in, 26, hours, using, ...","[closed, deal, 26, hours, using, AdobeSign, ht...","[close, deal, 26, hour, use, adobesign, httpst...","[close, deal, 26, hour, use, adobesign, httpst..."
4,just closed a deal in 6 days using #AdobeSign ...,"[just, closed, a, deal, in, 6, days, using, #,...","[closed, deal, 6, days, using, AdobeSign, http...","[close, deal, 6, day, use, adobesign, httpstco...","[close, deal, 6, day, use, adobesign, httpstco..."
5,just closed a deal in 35 minutes using #AdobeS...,"[just, closed, a, deal, in, 35, minutes, using...","[closed, deal, 35, minutes, using, AdobeSign, ...","[close, deal, 35, minut, use, adobesign, https...","[close, deal, 35, minut, use, adobesign, https..."
6,just closed a deal in 3 days using #AdobeSign ...,"[just, closed, a, deal, in, 3, days, using, #,...","[closed, deal, 3, days, using, AdobeSign, http...","[close, deal, 3, day, use, adobesign, httpstco...","[close, deal, 3, day, use, adobesign, httpstco..."
7,just closed a deal in 58 minutes using #AdobeS...,"[just, closed, a, deal, in, 58, minutes, using...","[closed, deal, 58, minutes, using, AdobeSign, ...","[close, deal, 58, minut, use, adobesign, https...","[close, deal, 58, minut, use, adobesign, https..."
8,just closed a deal in 7 minutes using #AdobeSi...,"[just, closed, a, deal, in, 7, minutes, using,...","[closed, deal, 7, minutes, using, AdobeSign, h...","[close, deal, 7, minut, use, adobesign, httpst...","[close, deal, 7, minut, use, adobesign, httpst..."
9,So apparently IU is catching on and wonTMt jus...,"[So, apparently, IU, is, catching, on, and, wo...","[apparently, IU, catching, wonTMt, show, new, ...","[appar, IU, catch, wontmt, show, new, serial, ...","[appar, IU, catch, wontmt, show, new, serial, ..."


In [37]:
df2 = feature_engineering(df)
df3 = df2

/home/daniel/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df3['time_created'] = pd.to_datetime(df3['created_at']).copy()
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

Index(['text', 'spam', 'spam_marketing', 'spam_hijack', 'spam_corporate',
       'spam_bot', 'spam_known', 'spam_own', 'Docusign', 'onespan', 'signnow',
       'adobe sign', 'listed_count', 'description', 'statuses_count',
       'followers_count', 'favourites_count', 'friends_count',
       'time_float_sin', 'time_float_cos', 'is_description_none'],
      dtype='object')

In [38]:
text_tfidf_matrix = to_tfidf(df3,'text',1000)
desc_tfidf_matrix = to_tfidf(df3, 'description', 1000)
col_name_lst = []
for n in range(text_tfidf_matrix.shape[1]):
    col_name = f'text-TF-IDF-{n}'
    df3[col_name] = text_tfidf_matrix[:, n]
    col_name_lst.append(col_name)
col_name_lst = []
for n in range(desc_tfidf_matrix.shape[1]):
    col_name = f'desc-TF-IDF-{n}'
    df3[col_name] = desc_tfidf_matrix[:, n]
    col_name_lst.append(col_name)

In [39]:
df3.columns

Index(['text', 'spam', 'spam_marketing', 'spam_hijack', 'spam_corporate',
       'spam_bot', 'spam_known', 'spam_own', 'Docusign', 'onespan',
       ...
       'desc-TF-IDF-990', 'desc-TF-IDF-991', 'desc-TF-IDF-992',
       'desc-TF-IDF-993', 'desc-TF-IDF-994', 'desc-TF-IDF-995',
       'desc-TF-IDF-996', 'desc-TF-IDF-997', 'desc-TF-IDF-998',
       'desc-TF-IDF-999'],
      dtype='object', length=2026)

In [9]:
df2 = feature_engineering(df)
df3 = df2
data_training_set = df3[:16000]
data_test_set = df3[16000:]
y = data_training_set['spam_marketing']
X = data_training_set[['Docusign', 'onespan', 'signnow','adobe sign','listed_count', 'statuses_count','followers_count',
          'favourites_count', 'friends_count','time_float_sin','time_float_cos', 'is_description_none']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
model = RandomForestClassifier()
model.fit(X_train,y_train)
y_predict = model.predict(X_test)
y_pred = model.predict_proba(X_test)
print("Accuracy score of this model is" + "\n")
print(accuracy_score(y_test,y_predict))
print("\n" + "Confusion Matrix of this model is" + "\n")
print(confusion_matrix(y_test,y_predict))
print("\n" + "Log-Loss score of this model is" + "\n")
print(log_loss(y_test,y_pred))
print("\n" + "AUC score of this model is" + "\n")
print(roc_auc_score(y_test,y_predict))
y = data_test_set['spam_marketing']
X = data_test_set[['Docusign', 'onespan', 'signnow','adobe sign','listed_count', 'statuses_count','followers_count',
          'favourites_count', 'friends_count','time_float_sin','time_float_cos', 'is_description_none']]
y_predict = model.predict(X)
y_pred = model.predict_proba(X)
print("Accuracy score of this model is" + "\n")
print(accuracy_score(y,y_predict))
print("\n" + "Confusion Matrix of this model is" + "\n")
print(confusion_matrix(y,y_predict))
print("\n" + "Log-Loss score of this model is" + "\n")
print(log_loss(y,y_pred))
print("\n" + "AUC score of this model is" + "\n")
print(roc_auc_score(y,y_predict))

Accuracy score of this model is

0.95875

Confusion Matrix of this model is

[[3771   42]
 [ 123   64]]

Log-Loss score of this model is

0.3650547300290184

AUC score of this model is

0.6656155202228234
Accuracy score of this model is

0.8699450823764353

Confusion Matrix of this model is

[[3443  133]
 [ 388   42]]

Log-Loss score of this model is

0.7253016293173199

AUC score of this model is

0.5302410124343166


/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [4]:
def testing_with_text(df2,b,k):
    for n in range(b,k,1000):
        df3 = df2
        text_tfidf_matrix = to_tfidf(df3,'text',n)
        col_name_lst = []
        for m in range(text_tfidf_matrix.shape[1]):
            col_name = f'text-TF-IDF-{m}'
            df3[col_name] = text_tfidf_matrix[:, m]
            col_name_lst.append(col_name)
        data_training_set = df3[:16000]
        data_test_set = df3[16000:]
        y = data_training_set['spam_marketing']
        X = data_training_set[['Docusign', 'onespan', 'signnow','adobe sign','listed_count', 'statuses_count','followers_count',
          'favourites_count', 'friends_count','time_float_sin','time_float_cos', 'is_description_none']+col_name_lst]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
        model = RandomForestClassifier()
        model.fit(X_train,y_train)
        y_predict = model.predict(X_test)
        y_pred = model.predict_proba(X_test)                
        print("\n" + "Result of " + str(n) + " text features with training set")
        print("Accuracy score of this model is" + "\n")
        print(accuracy_score(y_test,y_predict))
        print("\n" + "Confusion Matrix of this model is" + "\n")
        print(confusion_matrix(y_test,y_predict))
        print("\n" + "Log-Loss score of this model is" + "\n")
        print(log_loss(y_test,y_pred))
        print("\n" + "AUC score of this model is" + "\n")
        print(roc_auc_score(y_test,y_predict))
        y = data_test_set['spam_marketing']
        X = data_test_set[['Docusign', 'onespan', 'signnow','adobe sign','listed_count', 'statuses_count','followers_count',
          'favourites_count', 'friends_count','time_float_sin','time_float_cos', 'is_description_none']+col_name_lst]
        y_predict = model.predict(X)
        y_pred = model.predict_proba(X)                
        print("\n" + "Result of " + str(n) + " text features with test set")
        print("Accuracy score of this model is" + "\n")
        print(accuracy_score(y,y_predict))
        print("\n" + "Confusion Matrix of this model is" + "\n")
        print(confusion_matrix(y,y_predict))
        print("\n" + "Log-Loss score of this model is" + "\n")
        print(log_loss(y,y_pred))
        print("\n" + "AUC score of this model is" + "\n")
        print(roc_auc_score(y,y_predict))

In [5]:
testing_with_text(df2,1000,6000)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Result of 1000 text features with training set
Accuracy score of this model is

0.96125

Confusion Matrix of this model is

[[3793   27]
 [ 128   52]]

Log-Loss score of this model is

0.3185801779671794

AUC score of this model is

0.6409104130308317

Result of 1000 text features with test set
Accuracy score of this model is

0.8267598602096855

Confusion Matrix of this model is

[[3270  306]
 [ 388   42]]

Log-Loss score of this model is

0.9438931951395637

AUC score of this model is

0.5060519744029968


/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Result of 2000 text features with training set
Accuracy score of this model is

0.962

Confusion Matrix of this model is

[[3796   19]
 [ 133   52]]

Log-Loss score of this model is

0.3447367840885868

AUC score of this model is

0.638050370160462

Result of 2000 text features with test set
Accuracy score of this model is

0.8387418871692461

Confusion Matrix of this model is

[[3293  283]
 [ 363   67]]

Log-Loss score of this model is

0.9706400046857735

AUC score of this model is

0.538337625513761


/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Result of 3000 text features with training set
Accuracy score of this model is

0.95925

Confusion Matrix of this model is

[[3782   16]
 [ 147   55]]

Log-Loss score of this model is

0.38715527978603337

AUC score of this model is

0.634032242086768

Result of 3000 text features with test set
Accuracy score of this model is

0.8841737393909136

Confusion Matrix of this model is

[[3510   66]
 [ 398   32]]

Log-Loss score of this model is

1.053449587047028

AUC score of this model is

0.5279811144061183


/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Result of 4000 text features with training set
Accuracy score of this model is

0.95875

Confusion Matrix of this model is

[[3787   19]
 [ 146   48]]

Log-Loss score of this model is

0.35539291557506103

AUC score of this model is

0.6212152813517452

Result of 4000 text features with test set
Accuracy score of this model is

0.8579630554168747

Confusion Matrix of this model is

[[3414  162]
 [ 407   23]]

Log-Loss score of this model is

1.067327304310931

AUC score of this model is

0.5040931793351022


/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Result of 5000 text features with training set
Accuracy score of this model is

0.95875

Confusion Matrix of this model is

[[3788   18]
 [ 147   47]]

Log-Loss score of this model is

0.2736169483792253

AUC score of this model is

0.6187693332827711

Result of 5000 text features with test set
Accuracy score of this model is

0.8704443334997504

Confusion Matrix of this model is

[[3448  128]
 [ 391   39]]

Log-Loss score of this model is

0.9630934324409527

AUC score of this model is

0.5274517454867071


In [15]:
df2.columns

Index(['text', 'spam', 'spam_marketing', 'spam_hijack', 'spam_corporate',
       'spam_bot', 'spam_known', 'spam_own', 'Docusign', 'onespan',
       ...
       'text-TF-IDF-4353', 'text-TF-IDF-4354', 'text-TF-IDF-4355',
       'text-TF-IDF-4356', 'text-TF-IDF-4357', 'text-TF-IDF-4358',
       'text-TF-IDF-4359', 'text-TF-IDF-4360', 'text-TF-IDF-4361',
       'text-TF-IDF-4362'],
      dtype='object', length=4389)

In [7]:
col_name_lst = []
for n in range(text_tfidf_matrix.shape[1]):
    col_name = f'text-TF-IDF-{n}'
    df2[col_name] = text_tfidf_matrix[:, n]
    col_name_lst.append(col_name)

In [9]:
data_training_set = df2[:16000]
data_test_set = df2[16000:]

In [10]:
y = data_training_set['spam_marketing']
X = data_training_set[['Docusign', 'onespan', 'signnow','adobe sign','listed_count', 'statuses_count','followers_count',
          'favourites_count', 'friends_count','time_float_sin','time_float_cos', 'is_description_none']+col_name_lst]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
model = RandomForestClassifier()
model.fit(X_train,y_train)
y_predict = model.predict(X_test)
y_pred = model.predict_proba(X_test)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [14]:
print("Accuracy score of this model is" + "\n")
print(accuracy_score(y_test,y_predict))
print("\n" + "Confusion Matrix of this model is" + "\n")
print(confusion_matrix(y_test,y_predict))
print("\n" + "Log-Loss score of this model is" + "\n")
print(log_loss(y_test,y_pred))
print("\n" + "AUC score of this model is" + "\n")
print(roc_auc_score(y_test,y_predict))

Accuracy score of this model is

0.9585

Confusion Matrix of this model is

[[3783   28]
 [ 138   51]]

Log-Loss score of this model is

0.3090738048665969

AUC score of this model is

0.6312470584315244


In [18]:
df2 = feature_engineering(df)

/home/daniel/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df3['time_created'] = pd.to_datetime(df3['created_at']).copy()
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [19]:
def testing_with_text(df2,n):
    df3 = df2
    text_tfidf_matrix = to_tfidf(df3,'text',1000)
    col_name_lst = []
    for n in range(text_tfidf_matrix.shape[1]):
        col_name = f'text-TF-IDF-{n}'
        df3[col_name] = text_tfidf_matrix[:, n]
        col_name_lst.append(col_name)
    data_training_set = df3[:16000]
    data_test_set = df3[16000:]
    y = data_training_set['spam_marketing']
    X = data_training_set[['Docusign', 'onespan', 'signnow','adobe sign','listed_count', 'statuses_count','followers_count',
          'favourites_count', 'friends_count','time_float_sin','time_float_cos', 'is_description_none']+col_name_lst]
    print("Accuracy score of this model is" + "\n")
    print(accuracy_score(y_test,y_predict))
    print("\n" + "Confusion Matrix of this model is" + "\n")
    print(confusion_matrix(y_test,y_predict))
    print("\n" + "Log-Loss score of this model is" + "\n")
    print(log_loss(y_test,y_pred))
    print("\n" + "AUC score of this model is" + "\n")
    print(roc_auc_score(y_test,y_predict))

Accuracy score of this model is

0.9585

Confusion Matrix of this model is

[[3783   28]
 [ 138   51]]

Log-Loss score of this model is

0.3090738048665969

AUC score of this model is

0.6312470584315244


Testing Random Forest for spam_marketing, up to 9000 post text features

In [26]:
def testing_with_text(df2,k):
    for n in range(1000,k,1000):
        df3 = df2
        text_tfidf_matrix = to_tfidf(df3,'text',n)
        col_name_lst = []
        for m in range(text_tfidf_matrix.shape[1]):
            col_name = f'text-TF-IDF-{m}'
            df3[col_name] = text_tfidf_matrix[:, m]
            col_name_lst.append(col_name)
        data_training_set = df3[:16000]
        data_test_set = df3[16000:]
        y = data_training_set['spam_marketing']
        X = data_training_set[['Docusign', 'onespan', 'signnow','adobe sign','listed_count', 'statuses_count','followers_count',
          'favourites_count', 'friends_count','time_float_sin','time_float_cos', 'is_description_none']+col_name_lst]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
        model = RandomForestClassifier()
        model.fit(X_train,y_train)
        y_predict = model.predict(X_test)
        y_pred = model.predict_proba(X_test)                
        print("\n" + "Result of " + str(n) + " text features")
        print("Accuracy score of this model is" + "\n")
        print(accuracy_score(y_test,y_predict))
        print("\n" + "Confusion Matrix of this model is" + "\n")
        print(confusion_matrix(y_test,y_predict))
        print("\n" + "Log-Loss score of this model is" + "\n")
        print(log_loss(y_test,y_pred))
        print("\n" + "AUC score of this model is" + "\n")
        print(roc_auc_score(y_test,y_predict))

In [ ]:
testing_with_text(df2,10000)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Result of 1000 text features
Accuracy score of this model is

0.961

Confusion Matrix of this model is

[[3787   26]
 [ 130   57]]

Log-Loss score of this model is

0.3688046525745117

AUC score of this model is

0.6489970281797004


/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Result of 2000 text features
Accuracy score of this model is

0.95975

Confusion Matrix of this model is

[[3783   27]
 [ 134   56]]

Log-Loss score of this model is

0.35174535269509977

AUC score of this model is

0.6438251139660174


/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


...


Result of 4000 text features
Accuracy score of this model is

0.95675

Confusion Matrix of this model is

[[3771   26]
 [ 147   56]]

Log-Loss score of this model is

0.43261897504259683

AUC score of this model is

0.6345072788862351


/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Result of 5000 text features
Accuracy score of this model is

0.95925

Confusion Matrix of this model is

[[3790   27]
 [ 136   47]]

Log-Loss score of this model is

0.3416721491316184

AUC score of this model is

0.6248784915341348


/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Result of 6000 text features
Accuracy score of this model is

0.9585

Confusion Matrix of this model is

[[3793   20]
 [ 146   41]]

Log-Loss score of this model is

0.4325030889022369

AUC score of this model is

0.6070030615779679


In [28]:
df3.columns

Index(['text', 'spam', 'spam_marketing', 'spam_hijack', 'spam_corporate',
       'spam_bot', 'spam_known', 'spam_own', 'Docusign', 'onespan',
       ...
       'text-TF-IDF-3990', 'text-TF-IDF-3991', 'text-TF-IDF-3992',
       'text-TF-IDF-3993', 'text-TF-IDF-3994', 'text-TF-IDF-3995',
       'text-TF-IDF-3996', 'text-TF-IDF-3997', 'text-TF-IDF-3998',
       'text-TF-IDF-3999'],
      dtype='object', length=4026)